In [ ]:
"""
story:
let's assume we have a LIDAR
https://www.nuscenes.org/data-annotation
https://level5.lyft.com/dataset/#data-format

WE have a lidar and a detector which should detect pedestrian/traffic signs/and free space
the detector was built by someone else and we do not know anything about its quality, but we want to (and have to) use this in practice.

We want to make the best out of it using repeated measurements.

It is imporant that we asusme that these measurements are completely independent form each other...which is not ture, but we will deal with this in practice later.

"""

import numpy as np

In [ ]:
"""
class Model():
  #this is a  simple probabilistic model representing our sensor
  def __init__ (self):
    #constructor

  def GenerateState(self):
    #this function generates a random hidden state
    #has no return value

  def GetState(self):
    #this is a function we can not use inn practice, it reveals the hidden state. We will use this to check our result.
    #returns a number, the index of the hidden state (0,1,2)

  def Measure(self):
    #this funciton makes a measuerement with a previously set hidden state
    #returns the index of the measured state (0,1,2)

  def MeasureKnownState(self,X):
    #this function makes a measurement with a known hidden state
    #the hidden state is the imnput aprametre of the function (0,1,2)
    #returns the index of the measured state (0,1,2)
"""

! wget https://users.itk.ppke.hu/~horan/LIDARModel.py
import LIDARModel as Model1

! wget https://users.itk.ppke.hu/~horan/LIDARModel2.py
import LIDARModel2 as Model2

--2022-11-02 06:58:59--  https://users.itk.ppke.hu/~horan/LIDARModel.py
Resolving users.itk.ppke.hu (users.itk.ppke.hu)... 193.225.109.33
Connecting to users.itk.ppke.hu (users.itk.ppke.hu)|193.225.109.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1095 (1.1K) [text/x-python]
Saving to: ‘LIDARModel.py’

LIDARModel.py       100%[===================>]   1.07K  --.-KB/s    in 0s      

2022-11-02 06:59:01 (94.9 MB/s) - ‘LIDARModel.py’ saved [1095/1095]

--2022-11-02 06:59:01--  https://users.itk.ppke.hu/~horan/LIDARModel2.py
Resolving users.itk.ppke.hu (users.itk.ppke.hu)... 193.225.109.33
Connecting to users.itk.ppke.hu (users.itk.ppke.hu)|193.225.109.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1101 (1.1K) [text/x-python]
Saving to: ‘LIDARModel2.py’

LIDARModel2.py      100%[===================>]   1.08K  --.-KB/s    in 0s      

2022-11-02 06:59:02 (124 MB/s) - ‘LIDARModel2.py’ saved [1101/1101]



In [ ]:
M1 = Model1.Model() #initalize this model

repeat = 5000
RelFreqMatrix = np.zeros((3,3))
for state in range(3):
    for i in range(repeat):
        RelFreqMatrix[state,M1.MeasureKnownState(state)] += 1

ObservationMatrix1 = RelFreqMatrix/repeat
print(RelFreqMatrix)
DecisionMatrix1 = (RelFreqMatrix/np.sum(RelFreqMatrix,0)).T



[[2537. 1051. 1412.]
 [1574. 2953.  473.]
 [ 519.  509. 3972.]]


In [ ]:
M2 = Model2.Model() #initalize this model

repeat = 5000
RelFreqMatrix = np.zeros((3,3))
for state in range(3):
    for i in range(repeat):
        RelFreqMatrix[state,M2.MeasureKnownState(state)] += 1

ObservationMatrix2 = RelFreqMatrix/repeat
print(RelFreqMatrix)
DecisionMatrix2= (RelFreqMatrix/np.sum(RelFreqMatrix,0)).T


[[   0. 4519.  481.]
 [1531. 2956.  513.]
 [ 509.  485. 4006.]]


In [ ]:
def Entropy(p):
  H=0
  for a in p:
    if a != 0:
      H+= -a*np.log2(a)
  return H

PX=np.array([1/3,1/3,1/3]).T
print(PX)

[0.33333333 0.33333333 0.33333333]


In [ ]:
PX.shape = (3,1)

In [ ]:
PX.shape

(3, 1)

In [ ]:
np.matmul(ObservationMatrix2.T,PX)

array([[0.136     ],
       [0.53066667],
       [0.33333333]])

In [ ]:
(PX.T*ObservationMatrix2)

array([[0.        , 0.30126667, 0.03206667],
       [0.10206667, 0.19706667, 0.0342    ],
       [0.03393333, 0.03233333, 0.26706667]])

In [ ]:
def JointDistribution(Z,X):
  #Z is a matrix and X is a vector
  return (Z.T*X).T

print(ObservationMatrix2)
print(ObservationMatrix2.T)
print(PX)
print(ObservationMatrix2.T*PX)
print(JointDistribution(ObservationMatrix2,PX))

[[0.     0.9038 0.0962]
 [0.3062 0.5912 0.1026]
 [0.1018 0.097  0.8012]]
[[0.     0.3062 0.1018]
 [0.9038 0.5912 0.097 ]
 [0.0962 0.1026 0.8012]]
[[0.33333333]
 [0.33333333]
 [0.33333333]]
[[0.         0.10206667 0.03393333]
 [0.30126667 0.19706667 0.03233333]
 [0.03206667 0.0342     0.26706667]]
[[0.         0.30126667 0.03206667]
 [0.10206667 0.19706667 0.0342    ]
 [0.03393333 0.03233333 0.26706667]]


In [ ]:
def MutualInformation(ObservationMatrix, Prior):
  #H(x)+H(Z)-H(X,Z)
  HX= Entropy(Prior)
  PXZ=JointDistribution(ObservationMatrix,Prior)
  ObsProbForTheCurrentPrior = np.sum(PXZ,0)

  HZ = Entropy(ObsProbForTheCurrentPrior)

  #HXZ = Entropy(PXZ) - this should be it but this function works only with vectors.
  HXZ = -(PXZ*np.log2(PXZ, out=np.zeros_like(PXZ), where = PXZ!=0)).sum()
  return HX+HZ-HXZ



print(MutualInformation(ObservationMatrix1,PX))



[0.33495526]


In [ ]:
PX = np.array([1/3,1/3,1/3]).T
for iteration in range(10):
  MI1 = MutualInformation(ObservationMatrix1,PX)
  MI2 = MutualInformation(ObservationMatrix2,PX)
  if MI1>MI2:
    PXZ=JointDistribution(ObservationMatrix1,PX)
    PZ = np.sum(PXZ,0)
    posteriro = ObservationMatrix1[:,M1.Measure()]*PX/PZ
  else:
    PXZ=JointDistribution(ObservationMatrix2,PX)
    PZ = np.sum(PXZ,0)
    posteriro = ObservationMatrix2[:,M2.Measure()]*PX/PZ

In [ ]:
OM = np.array([[0.45, 0.45, 0.1], [0.45, 0.45, 0.1], [0.1, 0.1, 0.8]])
P = np.array([0.33, 0.33, 0.33]).T
JD = JointDistribution(OM,P.T)
JD

array([[0.1485, 0.1485, 0.033 ],
       [0.1485, 0.1485, 0.033 ],
       [0.033 , 0.033 , 0.264 ]])

In [ ]:
HX= Entropy(P)
HX

1.5834674497121084

In [ ]:
np.sum(JD,0)

array([0.33, 0.33, 0.33])

In [ ]:
HZ = Entropy(np.sum(JD,0))
HZ

1.5834674497121086

In [ ]:
print(-(JD*np.log2(JD, out=np.zeros_like(JD), where = JD!=0)).sum())

2.791240812793864


In [ ]:
MutualInformation(OM, P)

0.3756940866303533